In [1]:
import pandas as pd

In [3]:
df = pd.read_csv('IMDB Dataset.csv')

In [4]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [5]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [6]:
df.dropna(inplace=True)

In [7]:
df['sentiment'].value_counts()

negative    25000
positive    25000
Name: sentiment, dtype: int64

In [8]:
df.shape

(50000, 2)

In [9]:
df["review"][3]

"Basically there's a family where a little boy (Jake) thinks there's a zombie in his closet & his parents are fighting all the time.<br /><br />This movie is slower than a soap opera... and suddenly, Jake decides to become Rambo and kill the zombie.<br /><br />OK, first of all when you're going to make a film you must Decide if its a thriller or a drama! As a drama the movie is watchable. Parents are divorcing & arguing like in real life. And then we have Jake with his closet which totally ruins all the film! I expected to see a BOOGEYMAN similar movie, and instead i watched a drama with some meaningless thriller spots.<br /><br />3 out of 10 just for the well playing parents & descent dialogs. As for the shots with Jake: just ignore them."

In [10]:
import seaborn as sns

sns.countplot(x='sentiment', data=df)

In [11]:
def preprocess_text(sen):
    # Removing html tags
    sentence = remove_tags(sen)

    # Remove punctuations and numbers
    sentence = re.sub('[^a-zA-Z]', ' ', sentence)

    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)

    # Removing multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)

    return sentence

In [12]:
import re
TAG_RE = re.compile(r'<[^>]+>')

def remove_tags(text):
    return TAG_RE.sub('', text)


In [13]:
X = []
sentences = list(df['review'])
for sen in sentences:
    X.append(preprocess_text(sen))

In [14]:
X[3]

'Basically there a family where little boy Jake thinks there a zombie in his closet his parents are fighting all the time This movie is slower than soap opera and suddenly Jake decides to become Rambo and kill the zombie OK first of all when you re going to make film you must Decide if its thriller or drama As drama the movie is watchable Parents are divorcing arguing like in real life And then we have Jake with his closet which totally ruins all the film expected to see BOOGEYMAN similar movie and instead watched drama with some meaningless thriller spots out of just for the well playing parents descent dialogs As for the shots with Jake just ignore them '

In [15]:
type(X)

list

In [16]:
df1 = pd.Series((v[0] for v in X))


In [17]:
df2 = pd.Series(X)

In [18]:
df2[3]

'Basically there a family where little boy Jake thinks there a zombie in his closet his parents are fighting all the time This movie is slower than soap opera and suddenly Jake decides to become Rambo and kill the zombie OK first of all when you re going to make film you must Decide if its thriller or drama As drama the movie is watchable Parents are divorcing arguing like in real life And then we have Jake with his closet which totally ruins all the film expected to see BOOGEYMAN similar movie and instead watched drama with some meaningless thriller spots out of just for the well playing parents descent dialogs As for the shots with Jake just ignore them '

In [19]:
X = df2

In [20]:
y = df['sentiment']

In [21]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [22]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()

In [23]:
X_train_counts = count_vect.fit_transform(X_train)
X_train_counts.shape

(33500, 85783)

In [24]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()

X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(33500, 85783)

In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()

X_train_tfidf = vectorizer.fit_transform(X_train) # remember to use the original X_train set
X_train_tfidf.shape

(33500, 85783)

In [26]:
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC


In [27]:
svc_model = LinearSVC()
svc_model.fit(X_train_tfidf,y_train)
# predictions = svc_model.predict(X_test)
# print(metrics.classification_report(y_test,predictions))
# print(metrics.accuracy_score(y_test,predictions))

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [28]:
text_clf = Pipeline([('tfidf', TfidfVectorizer()),
                     ('clf', LinearSVC()),
])

# Feed the training data through the pipeline
text_clf.fit(X_train, y_train)  

Pipeline(memory=None,
     steps=[('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,...ax_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0))])

In [29]:
predictions = text_clf.predict(X_test)

In [30]:
from sklearn import metrics


print(metrics.classification_report(y_test,predictions))

              precision    recall  f1-score   support

    negative       0.91      0.89      0.90      8208
    positive       0.89      0.91      0.90      8292

   micro avg       0.90      0.90      0.90     16500
   macro avg       0.90      0.90      0.90     16500
weighted avg       0.90      0.90      0.90     16500



In [34]:
def func1(dataset):
    df = pd.read_csv(dataset)
    df.drop(['web-scraper-order', 'web-scraper-start-url','captions'], axis=1,inplace=True)
    positive = []
    negative = []
    i = 0
    for i in range(0,10):
        
        #print("i is ",i)
        test = df.sample(n=50)
        for j in range(0,50):
            test2 = test.iloc[j]
            res = text_clf.predict(test2)
            #print(res)
            if res == 'positive':
                positive.append(res)
            elif res == 'negative':
                negative.append(res)
    #print(type(positive), " po is ",positive)
    #print(type(negative)," negative is ",negative)
    print("total positive reviews are ",len(positive))
    print("total negative reviews are ",len(negative))
    print("% of positive reviews are ", (len(positive)*2)/10)
    print("% of positive reviews are ", (len(negative)*2)/10)

        
            
        
        

In [88]:
func1('lord_of_thle_flies.csv')#imdb 163 df 163

total positive reviews are  257
total negative reviews are  243
% of positive reviews are  51.4
% of positive reviews are  48.6


In [89]:
func1('nims_island.csv')#imdb 121 df 121 

total positive reviews are  359
total negative reviews are  141
% of positive reviews are  71.8
% of positive reviews are  28.2


In [90]:
func1('six_days_seven_nights_reviews.csv') # imdb 218  df 217

total positive reviews are  291
total negative reviews are  209
% of positive reviews are  58.2
% of positive reviews are  41.8


In [91]:
func1('blue_lagoon.csv')#imdb 198 df 198

total positive reviews are  378
total negative reviews are  122
% of positive reviews are  75.6
% of positive reviews are  24.4


In [92]:
func1('the_most_dangerous_game.csv')#imdb 129 df 129

total positive reviews are  377
total negative reviews are  123
% of positive reviews are  75.4
% of positive reviews are  24.6


In [93]:
func1('the_red_turtle.csv') #imdb 134 df 132 

total positive reviews are  376
total negative reviews are  124
% of positive reviews are  75.2
% of positive reviews are  24.8


In [94]:
# #rankings - castaway
# 1]blue lagoon
# 2]the most dangerous game
# 3]the red turtle
# 4]nims island
# 5]six days seven nights
# 6]lord of the flies

In [95]:
def func2(dataset):
    df = pd.read_csv(dataset)
    df.drop(['web-scraper-order', 'web-scraper-start-url','captions'], axis=1,inplace=True)
    length = df.shape[0]
    positive = []
    negative = []
    for i in range(0,length):
        test2 = df.iloc[i]
        res = text_clf.predict(test2)
        #print(res)
        if res == 'positive':
            positive.append(res)
        elif res == 'negative':
            negative.append(res)
    #print(type(positive), " po is ",positive)
    #print(type(negative)," negative is ",negative)
    print("total length is ",length)
    print("total positive reviews are ",len(positive))
    print("total negative reviews are ",len(negative))
    print("% of positive reviews are ", (len(positive)/length)*100)
    print("% of negative reviews are ", (len(negative)/length)*100)

        
    

In [96]:
func2('lord_of_thle_flies.csv')

total length is  163
total positive reviews are  84
total negative reviews are  79
% of positive reviews are  51.533742331288344
% of negative reviews are  48.466257668711656


In [97]:
func2('nims_island.csv')

total length is  121
total positive reviews are  90
total negative reviews are  31
% of positive reviews are  74.3801652892562
% of negative reviews are  25.6198347107438


In [98]:
func2('six_days_seven_nights_reviews.csv')

total length is  216
total positive reviews are  125
total negative reviews are  91
% of positive reviews are  57.870370370370374
% of negative reviews are  42.129629629629626


In [99]:
func2('blue_lagoon.csv')

total length is  197
total positive reviews are  145
total negative reviews are  52
% of positive reviews are  73.60406091370558
% of negative reviews are  26.39593908629442


In [100]:
func2('the_most_dangerous_game.csv')

total length is  128
total positive reviews are  95
total negative reviews are  33
% of positive reviews are  74.21875
% of negative reviews are  25.78125


In [101]:
func2('the_red_turtle.csv')

total length is  131
total positive reviews are  96
total negative reviews are  35
% of positive reviews are  73.2824427480916
% of negative reviews are  26.717557251908396


In [ ]:
# rankings 
# 1] nims island
# 2] most dangerous game
# 3] blue lagoon
# 4] red turtle
# 5] 6 days 7 nights
# 6] lord of the flies